In [3]:
# Import modules
from keras.applications.inception_v3 import InceptionV3
from keras.models import load_model
from keras.layers import Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.utils import load_img
import glob

In [4]:
IMAGE_SIZE = [224, 224]
train_path = 'D:/Computer/Assignments/PlantDisease/Datasets/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
valid_path = 'D:/Computer/Assignments/PlantDisease/Datasets/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'

In [3]:
# Import the Inception model
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [4]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in inception.layers[:249]:
    layer.trainable = False
for layer in inception.layers[249:]:
    layer.trainable = True


In [5]:
folders = glob.glob('D:/Computer/Assignments/PlantDisease/Datasets/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/*')
len(folders)

38

In [4]:
model_finetuned = Sequential([
    inception,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(rate=0.2),
    Dense(38, activation='softmax')
])

NameError: name 'inception' is not defined

In [4]:
train_datagen = ImageDataGenerator( rescale=1./255, rotation_range=10, shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_set = train_datagen.flow_from_directory('D:/Computer/Assignments/PlantDisease/Datasets/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train',target_size=IMAGE_SIZE,batch_size=50,class_mode='categorical')

Found 140590 images belonging to 38 classes.


In [6]:
test_set = test_datagen.flow_from_directory('D:/Computer/Assignments/PlantDisease/Datasets/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid',target_size=IMAGE_SIZE,batch_size=50,class_mode='categorical')

Found 35144 images belonging to 38 classes.


In [10]:
# compile the model
model_finetuned.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

model_finetuned.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 flatten (Flatten)           (None, 51200)             0         
                                                                 
 dense (Dense)               (None, 512)               26214912  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 38)                19494     
                                                                 
Total params: 48,037,190
Trainable params: 37,349,286
Non-trainable params: 10,687,904
_________________________________________________________________


In [11]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

file_path = 'model_finetuned.h5'
checkpoint1 = ModelCheckpoint(file_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_accuracy", mode="max", patience=15)
callbacks_list = [checkpoint1, early] #early


In [12]:
his1 = model_finetuned.fit(train_set, validation_data = test_set, epochs = 10, steps_per_epoch = len(train_set), validation_steps = len(test_set), callbacks = callbacks_list)

Epoch 1/10
2812/2812 [==============================] - ETA: 0s - loss: 0.2884 - accuracy: 0.9201
Epoch 1: val_accuracy improved from -inf to 0.95487, saving model to model_finetuned.h5
2812/2812 [==============================] - 1403s 493ms/step - loss: 0.2884 - accuracy: 0.9201 - val_loss: 0.1410 - val_accuracy: 0.9549
Epoch 2/10
2812/2812 [==============================] - ETA: 0s - loss: 0.1019 - accuracy: 0.9682
Epoch 2: val_accuracy improved from 0.95487 to 0.95726, saving model to model_finetuned.h5
2812/2812 [==============================] - 1313s 467ms/step - loss: 0.1019 - accuracy: 0.9682 - val_loss: 0.1618 - val_accuracy: 0.9573
Epoch 3/10
2812/2812 [==============================] - ETA: 0s - loss: 0.0744 - accuracy: 0.9770
Epoch 3: val_accuracy improved from 0.95726 to 0.97872, saving model to model_finetuned.h5
2812/2812 [==============================] - 1310s 466ms/step - loss: 0.0744 - accuracy: 0.9770 - val_loss: 0.0657 - val_accuracy: 0.9787
Epoch 4/10
2812/2812 [

In [13]:
import pickle as pkl
with open('historyDict.txt', 'wb') as file_pikl:
    pkl.dump(his1.history, file_pikl)

In [6]:
model1 = load_model('model_finetuned.h5')


In [7]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 flatten (Flatten)           (None, 51200)             0         
                                                                 
 dense (Dense)               (None, 512)               26214912  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 38)                19494     
                                                                 
Total params: 48,037,190
Trainable params: 37,349,286
Non-trainable params: 10,687,904
_________________________________________________________________


In [58]:
import cv2
import numpy as np
f = 'D:/Computer/Assignments/PlantDisease/Datasets/test/test/PeachBacterialSpot.jpg'
img = cv2.imread(str(f))
img = cv2.resize(img, (224, 224))
img = img/255



In [59]:
p = model1.predict(np.asarray([img]))
idx = np.argmax(p)

1/1 [==============================] - 0s 29ms/step


In [60]:
classes=['Apple scab', 'Apple Black rot', 'Cedar apple rust', 
         'Apple healthy', 'Blueberry healthy', 
         'Cherry Powdery mildew', 'Cherry healthy', 
         'Corn Cercospora leaf spot', 'Corn Common rust', 
         'Corn Northern Leaf Blight', 'Corn healthy', 
         'Grape Black rot', 'Grape Black Measles', 
         'Grape Leaf blight', 'Grape healthy', 
         'Orange Haunglongbing', 'Peach Bacterial spot', 
         'Peach healthy', 'Bell Peppers Bacterial spot', 'Bell Peppers healthy', 
         'Potato Early blight', 'Potato Late blight', 'Potato healthy', 
         'Raspberry healthy', 'Soybean healthy', 'Squash Powdery mildew', 
         'Strawberry Leaf scorch', 'Strawberry healthy', 'Tomato Bacterial spot', 
         'Tomato Early blight', 'Tomato Late blight', 'Tomato Leaf Mold', 
         'Tomato Septoria leaf spot', 'Tomato Spider mites', 
         'Tomato Target Spot', 'Tomato Yellow Leaf Curl Virus', 
         'Tomato mosaic virus', 'Tomato healthy']


In [61]:
import matplotlib.pyplot as plt
img1 = cv2.imread(str(f))
cv2.putText(img1, classes[idx], (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
cv2.imshow('image', img1)
cv2.waitKey(0)

-1